# Autoencoders for images

__Objective:__ understand how to encode images in a 2-dimensional latent space via an autoencoder model.

In [ ]:
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Reshape, Conv2DTranspose
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('../../modules/')

from keras_utilities import append_to_full_history, plot_history

sns.set_theme()

## Load and preprocess data

In [ ]:
def preprocess_images(img):
    """
    """
    # Normalize pixel values.
    img = img.astype('float32') / 255.

    # Add padding.
    img = np.pad(img, ((0, 0), (2, 2), (2, 2)), constant_values=0.)
    
    # The images come in grayscale without an explicit
    # channels dimensions. Here we add it.
    img = np.expand_dims(img, -1)

    return img

In [ ]:
# Note: we don't really care about the labels in the y arrays.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(14, 6))

for i in range(len(axs)):
    axs[i].imshow(
        x_train[i, ...],
        cmap='gray'
    )
    
    axs[i].grid(False)

In [ ]:
x_train = preprocess_images(x_train)
x_test = preprocess_images(x_test)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(14, 6))

for i in range(len(axs)):
    axs[i].imshow(
        x_train[i, ...],
        cmap='gray'
    )
    
    axs[i].grid(False)

## Dutoencoder model

In [ ]:
def get_intermediate_output(x, model, n_layers):
    """
    Given the NN `model`, passes the input `x` through
    it first `n_layers` layers and outputs the result.
    """
    output = x
    
    for i in range(n_layers):
        output = model.layers[i](output)

    return output

Model definition.

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super().__init__()

        # Sequence of number of filters (for the corresponding
        # convolutional layers in the stack).
        self.filters_list = [32, 64, 128]

        # Input placeholder. Actually not needed if defining
        # the model this way.
        self.model_input = Input(shape=(32, 32, 1))

        # Stack of convolutional layers.
        self.conv_layers = [
            Conv2D(
                filters=filters,
                kernel_size=(3, 3),
                strides=2,
                activation='relu',
                padding='same'
            )
            for filters in self.filters_list
        ]

        # Flattening layer to reshape the image into a rank-1
        # tensor.
        self.flatten = Flatten()

        # Final dense layer, projecting the intermediate outputs
        # to the 2-dimensional latent space.
        self.final_dense = Dense(units=2)

    def call(self, x):
        for conv_layer in self.conv_layers:
            x = conv_layer(x)

        x = self.flatten(x)

        x = self.final_dense(x)

        return x


class Decoder(tf.keras.Model):
    def __init__(self, image_reshaping_size):
        super().__init__()

        # Initial dense layer: maps latent vectors into
        # vectors with enough components to be reshaped
        # into images.
        self.initial_dense = Dense(
            units=np.prod(image_reshaping_size)
        )

        # Reshapes a rank-1 vector into a rank-3 tensor
        # representing an image.
        self.reshape = Reshape(image_reshaping_size)

        # List of numbers filters to be used by the stack of
        # transposed convolutional layers. The length of the list
        # corresponds to the number of layers in the stack.
        self.filters_list = [128, 64, 32]

        # Stack of transposed convolutional layers.
        self.conv_transp_layers = [
            Conv2DTranspose(
                filters=filters,
                kernel_size=(3, 3),
                strides=2,
                activation='relu',
                padding='same'
            )
            for filters in self.filters_list
        ]

        # Final convolutional layer, outputting a tensor with
        # the right shape, one channel dimension (grayscale)
        # and pixel intensities normalized in [0, 1] (sigmoid
        # activation).
        self.final_conv = Conv2D(
            filters=1,
            kernel_size=(3, 3),
            strides=1,
            activation='sigmoid',
            padding='same'
        )

    def call(self, x):
        x = self.initial_dense(x)
        x = self.reshape(x)

        for conv_t_layer in self.conv_transp_layers:
            x = conv_t_layer(x)

        x = self.final_conv(x)

        return x

In [ ]:
encoder = Encoder()

encoder(x_train[:10]).shape

In [ ]:
# Last image-like shape (rank-3 tensor) before the flattening
# layer in the encoder.
image_reshaping_size = tuple(get_intermediate_output(x_train[:1, ...], encoder, 3).shape[1:])

decoder = Decoder(image_reshaping_size)

decoder(encoder(x_train[:2, ...])).shape

In [ ]:
autoencoder_input = tf.keras.Input(shape=x_train[0, ...].shape)

autoencoder_model = tf.keras.Model(
    inputs=autoencoder_input,
    outputs=decoder(encoder(autoencoder_input))
)

In [ ]:
autoencoder_model(x_train[:10, ...]).shape

Model training.

In [ ]:
autoencoder_model.compile(
    optimizer='adam',
    loss='binary_crossentropy'
)

full_history = dict([])

In [ ]:
epochs = 5

history = autoencoder_model.fit(
    x=x_train,
    y=x_train,
    epochs=epochs,
    batch_size=100,
    shuffle=True,
    validation_data=(x_test, x_test)
)

append_to_full_history(history, full_history)

In [ ]:
plot_history(full_history)

## Image reconstruction after training

In [ ]:
nrows = 2
ncols = 6

reconstructed_images = tf.concat(
    [
        x_test[:ncols, ...][None, ...],
        autoencoder_model(x_test[:ncols, ...])[None, ...]
    ],
    axis=0
)

fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(14, 4))

for i in range(nrows):
    for j in range(ncols):
        ax = axs[i][j]
        
        ax.imshow(
            reconstructed_images[i, j, ...],
            cmap='gray'
        )

        ax.grid(False)

## Exploration of the latent space

In [ ]:
y_test

In [ ]:
n_samples = 5000

latent_vectors = encoder(x_test[:n_samples, ...])

fig = plt.figure(figsize=(14, 6))

sns.scatterplot(
    x=latent_vectors[:, 0],
    y=latent_vectors[:, 1],
    hue=y_test[:n_samples],
    palette=sns.color_palette()
)

## Generating new images

**Idea:** we randomly sample the latent space and have the decoder produce an image from the new latent vectors.

In [ ]:
n_samples = 6

# Bounds (along each dimension of the latent space)
# of the region of latent space we want to randomly
# sample from (a rectangle).
bounds = ((-5., 0.), (-5., 10.))

# Random 2-dimensional vectors in the chose region
# in latent space.
random_latent_vectors = tf.concat(
    [
        tf.random.uniform(shape=(n_samples, 1), minval=bounds[0][0], maxval=bounds[0][1]),
        tf.random.uniform(shape=(n_samples, 1), minval=bounds[1][0], maxval=bounds[1][1])
    ],
    axis=-1
)

# Decode the randomly-generated latent vectors into
# images via the decoder.
random_images = decoder(random_latent_vectors)


# Plot the position of the random latent vectors over
# existing samples.
fig = plt.figure(figsize=(14, 6))

sns.scatterplot(
    x=latent_vectors[:, 0],
    y=latent_vectors[:, 1],
    color=sns.color_palette()[0],
    alpha=.3
)

sns.scatterplot(
    x=random_latent_vectors[:, 0],
    y=random_latent_vectors[:, 1],
    color=sns.color_palette()[3],
)


# Show the decoded images corresponding to the random
# latent vectors.
fig, axs = plt.subplots(ncols=n_samples, figsize=(14, 4))

for i in range(n_samples):
    axs[i].imshow(
        random_images[i, ...],
        cmap='gray'
    )

    axs[i].grid(False)

    plt.sca(axs[i])
    plt.title(f'{random_latent_vectors[i, ...].numpy().round(2)}')

Consider a path (straight line) in latent space and generate samples going along it to see how they change. This should generate some kind of morphing between the images corresponding to the initial and final points, but nothing guarantees that it will be continuous nor that all the generated images will be realistic!

In [ ]:
# Generate path in latent space.
n_points = 20

starting_point = tf.constant([-5., -2.5])
endpoint = tf.constant([-2.5, 2.5])

path = (endpoint - starting_point) * tf.linspace(0., 1., n_points)[..., None] + starting_point

# Generate images correponding to points along
# the path.
images_along_path = decoder(path)

# Plot generated images.
ncols = 10
nrows = n_points // ncols

fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(14, 4))

for i in range(nrows):
    for j in range(ncols):
        ax = axs[i][j]
        
        ax.imshow(
            images_along_path[i * ncols + j, ...],
            cmap='gray'
        )

        ax.grid(False)

        plt.sca(ax)
        plt.xticks([])
        plt.yticks([])
        plt.title(f'{path[i * ncols + j, ...].numpy().round(2)}')